# Few-Shot and Chain-of-Thought Prompting

Let's consider some examples of how model output changes depending on whether we prompt by:
* Providing examples (zero-shot vs few-shot)
* Telling the model to reason, e.g. step-by-step for chain-of-thought

In [ ]:
import os
import json
from openai import OpenAI 
NRP_TOK = os.environ.get('NRP_TOK')
client = OpenAI(api_key = NRP_TOK, base_url = "https://ellm.nrp-nautilus.io/v1")

In [ ]:
MODEL='gpt-oss'

In [ ]:
def call_llm(user_prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    return response.choices[0].message.content

## Zero-shot vs Few-shot

Zero-shot prompting involves giving a task to a model without any examples, relying solely on its pre-trained knowledge to generate a response. 

Few-shot prompting enhances performance by including a small number of examples in the prompt, helping the model understand the task's pattern or format. 

While zero-shot is simpler and faster, few-shot typically yields more accurate and consistent results by providing contextual guidance.

In [ ]:
prompt_template_1 = '''
Classify the sentiment of the review.

Review:
{review}
'''

In [ ]:
review_text = [
    'you probably like this movie, but i did not.',
    'loved it',
    'it made my eyes glaze over',
    'it made me teary eyed'
]
for i in review_text:
    print(i)
    review_prompt = prompt_template_1.format(review=i)
    result = call_llm(review_prompt)
    print(result)

In [ ]:
prompt_template_2 = '''
Classify the sentiment of the review as "positive", "neutral", or "negative".

Review:
{review}

Answer with one word: positive, neutral, or negative.
'''

In [ ]:
review_text = [
    'you probably like this movie, but i did not.',
    'i loved it intensely',
    'it made my eyes glaze over',
    'it made me teary eyed'
]
for i in review_text:
    print(i)
    review_prompt = prompt_template_2.format(review=i)
    result = call_llm(review_prompt)
    print(result)

In [ ]:
prompt_template_3 = '''
Classify the sentiment of the review as "positive", "neutral", or "negative".

Example:
Review: 'you probably like this movie, but i did not.'
Sentiment: negative

Example:
Review: 'i loved it intensely'
Sentiment: positive

Example:
Review: 'it made me teary eyed'
Sentiment: neutral

Output:
Answer with one word: positive, neutral, or negative.

Review:
{review}
'''

In [ ]:
review_text = [
    'you probably like this movie, but i did not.',
    'i loved it intensely',
    'it made my eyes glaze over',
    'it made me teary eyed',
    'i cried'
]
for i in review_text:
    print(i)
    review_prompt = prompt_template_3.format(review=i)
    result = call_llm(review_prompt)
    print(result)

## Chain-of-Thought

Chain-of-thought prompting enhances LLM reasoning by guiding them to break down complex problems into smaller, logical steps before arriving at a final answer.  It works best with larger models and is particularly effective when the model is prompted with examples or clear instructions to think step by step.

In [ ]:
prompt_template_4 = '''
You are a movie critic. Classify the sentiment of the review 
as "positive", "neutral", or "negative". Show your reasoning
step-by-step.

Review:
{review}
'''

In [ ]:
review_text = [
    'you probably like this movie, but i did not.',
    'i loved it intensely',
    'it made my eyes glaze over',
    'it made me teary eyed',
    'i cried'
]
for i in review_text:
    print(i)
    review_prompt = prompt_template_4.format(review=i)
    result = call_llm(review_prompt)
    print(result)

This method can be especially useful for:
* Word problems and logic puzzles
* Complex instructions with many constraints (“Write code that does X, Y, Z but not W”)
* Planning problems (trip plans, project plans, multi-step workflows)
* Refactoring or debugging code (explain issues → then propose fix)

The technique can help to:
* Reduce the probability of "magic answers".
* Improve accuracy on multi-step problems (math, logic, planning, code, multi-constraint instructions).
* Make debugging easier: If the model is wrong, you can see where reasoning went off the rails.
* Educate: you see intermediate steps.

In [ ]:
life_planner = '''
You are a life coach and college counselor.

Task:
Help an undergraduate student choose between two career tracks.

Steps:
1. List pros and cons of approach A.
2. List pros and cons of approach B.
3. State your recommendation and explain in 2–3 sentences.

Approach A:
{track1}

Approach B:
{track2}

Format:
- Plain text.
- No Markdown.

Answer:
'''

In [ ]:
tracks_prompt = life_planner.format(track1='novelist',
                                    track2='AI engineer')
result = call_llm(tracks_prompt)
print(result)